# Credit Risk Resampling Techniques

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.tail()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
77531,19100.0,11.261,own,86600,0.653580,12,2,56600,high_risk
77532,17700.0,10.662,mortgage,80900,0.629172,11,2,50900,high_risk
77533,17600.0,10.595,rent,80300,0.626401,11,2,50300,high_risk
77534,16300.0,10.068,mortgage,75300,0.601594,10,2,45300,high_risk
77535,15600.0,9.742,mortgage,72300,0.585062,9,2,42300,high_risk


# Split the Data into Training and Testing

In [4]:
# figure out what I am trying to predict and then X becomes all the values - what we are trying to predict (Y)
# Create our features
X = df.drop(columns= ['loan_status'])
# using get_dummies to convert string data to numerical data (creates new column for each representation)
# needs to be done before the test train split so that this info. is captured by the model 
X=pd.get_dummies(X)

# Create our target (what we want to predict)
y = df.loan_status 


In [5]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [6]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [7]:
#library need to traintestsplit 
from sklearn.model_selection import train_test_split

# Create X_train, X_test, y_train, y_test
X_train,X_test, y_train, y_test = train_test_split(X,y)

#.shape allows me to see the data to have a better understanding of how the model is being trainned 
X_train.shape

(58152, 10)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [8]:
# Create the StandardScaler instance
# normalizing the data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# .fit is responsible for the normalization calculations
scaler.fit(X_train)
X_train

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
37503,9700.0,7.255,48900,0.386503,4,0,18900,1,0,0
14574,9700.0,7.263,48900,0.386503,4,0,18900,1,0,0
14541,8800.0,6.873,45300,0.337748,3,0,15300,0,1,0
621,9300.0,7.074,47200,0.364407,3,0,17200,0,1,0
16320,9000.0,6.951,46000,0.347826,3,0,16000,0,1,0
...,...,...,...,...,...,...,...,...,...,...
2977,10100.0,7.410,50300,0.403579,4,1,20300,0,1,0
18450,9600.0,7.201,48400,0.380165,4,0,18400,0,0,1
1923,9200.0,7.022,46700,0.357602,3,0,16700,0,1,0
31711,11600.0,8.035,56200,0.466192,5,1,26200,0,1,0


In [10]:
# Scale the training and testing data
# we only scale the X data because the X data has such a range of information (ie. multiple columns)
# .transform excutes the normalized calulations 
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Simple Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [12]:
# this is to help me visualize what is happening with the data 
Counter(y_train)

Counter({'low_risk': 56275, 'high_risk': 1877})

In [13]:
# Calculated the balanced accuracy score
# Testing the actual data that I have reserved to test aka. X_test 
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9866616507400731

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

# looking at data to see how the actual data did against my predicted data 
confusion_matrix(y_test, y_pred)

array([[  610,    13],
       [  109, 18652]])

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.98      0.99      0.91      0.99      0.97       623
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     18761

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(X_train_scaled,y_train)
# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56275, 'high_risk': 56275})

In [17]:
# printing to see if there is a difference in the data (for my own learning)
print(y_train)

37503    low_risk
14574    low_risk
14541    low_risk
621      low_risk
16320    low_risk
           ...   
2977     low_risk
18450    low_risk
1923     low_risk
31711    low_risk
41997    low_risk
Name: loan_status, Length: 58152, dtype: object


In [18]:
print(y_resampled)

0          low_risk
1          low_risk
2          low_risk
3          low_risk
4          low_risk
            ...    
112545    high_risk
112546    high_risk
112547    high_risk
112548    high_risk
112549    high_risk
Name: loan_status, Length: 112550, dtype: object


In [19]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
# Creating the LogisticRegression oversampled model 
oversampled_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
# insering the resampled data 
oversampled_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [20]:
# Calculated the balanced accuracy score using the resampled data 
resampled_ypred = oversampled_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, resampled_ypred)

0.9943408695149247

In [21]:
# Display the confusion matrix
confusion_matrix(y_test, resampled_ypred)

array([[  620,     3],
       [  122, 18639]])

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, resampled_ypred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      0.99      0.99       623
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18761

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [23]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled2, y_resampled2 = SMOTE(random_state =1,sampling_strategy=1.0).fit_resample(X_train_scaled,y_train)
# View the count of target classes with Counter
Counter(y_resampled2)

Counter({'low_risk': 56275, 'high_risk': 56275})

In [24]:
# Train the Logistic Regression model using the resampled data
LRM = LogisticRegression(solver='lbfgs', random_state =1)
LRM.fit(X_resampled2,y_resampled2)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
resampled_y2pred = LRM.predict(X_test_scaled)
balanced_accuracy_score(y_test,resampled_y2pred)


0.9943941715777145

In [26]:
# Display the confusion matrix
confusion_matrix(y_test,resampled_y2pred)

array([[  620,     3],
       [  120, 18641]])

In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,resampled_y2pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      0.99      0.99       623
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18761

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the data using the ClusterCentroids resampler
# importing the necessary library 
from imblearn.under_sampling import ClusterCentroids
# Creating the CC model 
cc = ClusterCentroids(random_state = 1)
# resampling the data 
cc_Xresampled, cc_yresampled = cc.fit_resample(X_train_scaled, y_train)
# View the count of target classes with Counter
Counter(cc_yresampled)

Counter({'high_risk': 1877, 'low_risk': 1877})

In [29]:
# Train the Logistic Regression model using the CC resampled data
cc_model = LogisticRegression(solver ='lbfgs',random_state = 1)
cc_model

LogisticRegression(random_state=1)

In [30]:
# fitting the model with resampled data 
cc_model.fit(cc_Xresampled,cc_yresampled)
# Calculating the prediction 
cc_ypred = cc_model.predict(X_test_scaled)
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, cc_ypred)

0.9914504090184695

In [31]:
# Display the confusion matrix with cc resampled data 
confusion_matrix(y_test,cc_ypred)

array([[  616,     7],
       [  110, 18651]])

In [32]:
# Print the imbalanced classification report with new cc resampled data 
print(classification_report_imbalanced(y_test,cc_ypred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.99      0.99      0.91      0.99      0.98       623
   low_risk       1.00      0.99      0.99      1.00      0.99      0.98     18761

avg / total       0.99      0.99      0.99      0.99      0.99      0.98     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Resample the training data with SMOTEENN
# import library 
from imblearn.combine import SMOTEENN
sm = SMOTEENN(random_state =1)
# resample the data using SMOTEENN
sm_Xresampled, sm_yresampled = sm.fit_resample(X_train_scaled, y_train)
# View the count of target classes with Counter
Counter(sm_yresampled)

Counter({'high_risk': 55670, 'low_risk': 55920})

In [34]:
# Train the Logistic Regression model using the resampled data
sm_model = LogisticRegression(solver ='lbfgs',random_state =1)
sm_model.fit(sm_Xresampled,sm_yresampled)
sm_model

LogisticRegression(random_state=1)

In [35]:
#creating a prediction variable 
sm_ypred = sm_model.predict(X_test_scaled)

# Calculate the balanced accuracy score with combined data 
balanced_accuracy_score(y_test,sm_ypred)

0.9943675205463196

In [36]:
# Display the confusion matrix
confusion_matrix(y_test,sm_ypred)

array([[  620,     3],
       [  121, 18640]])

In [37]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,sm_ypred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      0.99      0.99       623
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18761

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

A: The SMOTE oversampling model had the best balanced accuracy score, at 0.9655.

2. Which model had the best recall score?

A: The SMOTE model had the best recall score at 0.94. 

3. Which model had the best geometric mean score?

A: Once again, the SMOTE model has the best geometric mean score. 

